In [2]:
import os
import pandas as pd
from textblob import *
import nltk
import numpy as np
import openpyxl
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import *
'''
    WELFAKE
'''

data = pd.read_csv(r"../../data/fake_news_conventional/WELFAKE_PROCESSED.csv")
data["flag"] = data["label"]
data

print("WELFAKE LOADED")


data = data.sample(5000)
data

WELFAKE LOADED


,Unnamed: 0.1,Unnamed: 0,title,text,label,text_tb_pol,text_tb_sub,title_tb_pol,title_tb_sub,title_vader_neg,title_vader_neu,title_vader_pos,text_vader_neg,text_vader_neu,text_vader_pos,flag
49999,50392,50392,Human rights skirted at Asia summit as Trump r...,MANILA Reuters Leaders of Asian nations me...,0,0.005704,0.392649,0.000000,0.100000,0.000,1.000,0.000,0.083,0.820,0.097,0
29953,30164,30164,Track Palin Faces Assault And Weapons Charges...,It s no secret that the Palin family are fans ...,1,0.108910,0.454628,0.000000,0.100000,0.648,0.352,0.000,0.161,0.699,0.140,1
32155,32389,32389,WASHINGTON RESTAURANT Tells Local Sheriff Depu...,Except when someone tries to rob them or break...,1,0.255903,0.575347,0.000000,0.000000,0.000,1.000,0.000,0.063,0.844,0.093,1
34613,34872,34872,Now Begins our Most Significant Week in the Ma...,By Les Visible on November Reflections in a...,1,0.138408,0.557876,0.437500,0.687500,0.000,0.793,0.207,0.097,0.770,0.133,1
27155,27341,27341,New Zealand First leader says his party holds ...,WELLINGTON Reuters Populist New Zealand Fi...,0,0.092273,0.452576,0.174242,0.414141,0.000,0.838,0.162,0.000,0.827,0.173,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51331,51734,51734,Tillerson Palestinian Authority Intent On Cea...,The Times of Israel reports WASHINGTON US...,0,-0.027500,0.352768,0.000000,0.000000,0.331,0.565,0.105,0.190,0.778,0.032,0
60340,60839,60839,BOOM TRUMP POLL Numbers Going UpUpUp While M...,Way before the election ever took place the l...,1,0.101290,0.502723,-0.112500,0.062500,0.171,0.829,0.000,0.100,0.783,0.116,1
31046,31269,31269,VIDEO YEPGUN CONTROL BILL SAID THAT TODAY Y...,You can t have people walking around with gun...,1,-0.036790,0.422159,0.000000,0.000000,0.000,1.000,0.000,0.083,0.859,0.058,1
46789,47148,47148,HOLLYWOOD LIBS HAVE EPIC MELT DOWNSThreaten Pr...,Hollywood stars took to social media Wednesday...,1,0.035351,0.471157,0.000000,0.250000,0.000,1.000,0.000,0.172,0.725,0.103,1


In [3]:
'''
    Add tf-idf vectorizer
'''

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import pandas as pd


def get_tf_idf_vector(corpus: pd.Series) -> tuple[pd.DataFrame, list[str], TfidfVectorizer]:
    #temp = pd.concat((x, corpus), axis=1).reset_index()

    stop_words = stopwords.words("english")
    vectorizer = TfidfVectorizer(strip_accents="ascii", lowercase=True, stop_words=stop_words, max_features=500, ngram_range=(1,3))
    tf_idf_features = vectorizer.fit_transform(corpus).toarray()

    names = vectorizer.get_feature_names_out()
    headers = [f"__word{i}" for i in range(len(names))]
    feature_frame = pd.DataFrame(tf_idf_features, columns=headers).reset_index(drop=True)
    return (feature_frame.reset_index(drop=True), names, vectorizer)



In [4]:
from openpyxl import load_workbook, Workbook

'''
    Define filtered dataset, classes, features, dataframe for model accuracies, and excel file for results
'''

'''Drop NA'''
filtered = data.dropna()
classes = filtered["flag"]

filtered["corpus"] = filtered.apply(lambda x: " ".join([x["title"], x["text"]]), axis=1)


# raw polarity and subjectivity scores from Textblob, Vader
features = filtered[["text_tb_pol",	"text_tb_sub",	"title_tb_pol",	"title_tb_sub",	"title_vader_neg",	
              "title_vader_neu",	"title_vader_pos",	"text_vader_neg",	"text_vader_neu",	"text_vader_pos"]]



EXCEL_FILE = r"../../data/fake_news_conventional/WELFAKE_RESULTS.xlsx"
# overwrite book if exists
book = Workbook()
book.save(filename=EXCEL_FILE)
book.close()


In [5]:
'''
    POLARITY + TF-IDF
'''
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import KFold
import openpyxl.drawing
from sklearn import metrics
import matplotlib.pyplot as plt
import io



# average accuracies
comb_output = {"train" : None, "test" : None}
comb_matrices = []


comb_fpr = []
comb_tpr = []
comb_thresh = []

comb_auc = 0


train_total = 0
test_total = 0
run_count = 0


#lbgfs
x = features
y = classes
folds = 5
kf = KFold(n_splits=folds)

for i, (train_index, test_index) in enumerate(kf.split(x)):
    print(i)
    scaler = StandardScaler()
    print("tf_idf training")
    tf_idf_training, names, vectorizer = get_tf_idf_vector(filtered["corpus"].iloc[train_index])
    x_train = pd.concat((x.iloc[train_index].reset_index(drop=True), tf_idf_training), axis=1)
    
    x_train = scaler.fit_transform(x_train)

    print("tf_idf testing")
    x_test = pd.DataFrame(vectorizer.transform(filtered["corpus"].iloc[test_index]).toarray(), columns=[f"__word{i}" for i in range(len(names))])
    x_test = pd.concat((x.iloc[test_index].reset_index(drop=True), x_test), axis=1)
    x_test = scaler.transform(x_test)
    
   
    
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    comb_ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(kernel='rbf', probability=True, random_state=42))
    ], voting='soft')
    print("ensemble fitting")
    comb_ensemble.fit(x_train, y_train)
    y_pred = comb_ensemble.predict(x_test)

    fpr, tpr, thresh = metrics.roc_curve(y_test, y_pred, pos_label=1)
    comb_fpr.append(fpr)
    comb_tpr.append(tpr)
    comb_thresh.append(thresh)
    comb_auc += metrics.auc(fpr, tpr)

    comb_matrices.append(confusion_matrix(y_test, y_pred))
    test_total += accuracy_score(y_test, y_pred)

    y_train_pred = comb_ensemble.predict(x_train)
    train_total += accuracy_score(y_train, y_train_pred)

    run_count += 1

print(f"Train accuracy on {folds}-fold cross-validation: {train_total / float(run_count)}")
print(f"Test accuracy on {folds}-fold cross-validation: {test_total / float(run_count)}")

comb_output["test"] = [(test_total / run_count)]
comb_output["train"] = [(train_total / run_count)]
comb_output = pd.DataFrame(comb_output)

print("Average confusion matrix:")
comb_matrices = np.mean(np.array(comb_matrices), axis=0)
print(comb_matrices)


comb_fpr = np.mean(np.array(comb_fpr), axis=0)
comb_tpr = np.mean(np.array(comb_tpr), axis=0)
comb_thresh = np.mean(np.array(comb_thresh), axis=0)

comb_auc = comb_auc / float(run_count)


    


    


    

    
#x_test

comb_output

0
tf_idf training
tf_idf testing
ensemble fitting
1
tf_idf training
tf_idf testing
ensemble fitting
2
tf_idf training
tf_idf testing
ensemble fitting
3
tf_idf training
tf_idf testing
ensemble fitting
4
tf_idf training
tf_idf testing
ensemble fitting
Train accuracy on 5-fold cross-validation: 0.99605
Test accuracy on 5-fold cross-validation: 0.9272
Average confusion matrix:
[[451.6  40.2]
 [ 32.6 475.6]]


,train,test
0,0.99605,0.9272


In [6]:
'''
    POLARITY
'''
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import KFold


# average accuracies
pol_output = {"train" : None, "test" : None}
pol_matrices = []


pol_fpr = []
pol_tpr = []
pol_thresh = []

pol_auc = 0

train_total = 0
test_total = 0
run_count = 0


#lbgfs
x = features
y = classes
folds = 5
kf = KFold(n_splits=folds)

for i, (train_index, test_index) in enumerate(kf.split(x)):
    print(i)
    scaler = StandardScaler()
    print("tf_idf training")
    x_train = x.iloc[train_index]
    x_train = scaler.fit_transform(x_train)

    print("tf_idf testing")
    x_test = x.iloc[test_index]
    x_test = scaler.transform(x_test)
    
   
    
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    pol_ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(kernel='rbf', probability=True, random_state=42))
    ], voting='soft')
    print("ensemble fitting")
    pol_ensemble.fit(x_train, y_train)
    y_pred = pol_ensemble.predict(x_test)
    fpr, tpr, thresh = metrics.roc_curve(y_test, y_pred, pos_label=1)
    pol_fpr.append(fpr)
    pol_tpr.append(tpr)
    pol_thresh.append(thresh)
    pol_auc += metrics.auc(fpr, tpr)

    pol_matrices.append(confusion_matrix(y_test, y_pred))
    test_total += accuracy_score(y_test, y_pred)

    y_train_pred = pol_ensemble.predict(x_train)
    train_total += accuracy_score(y_train, y_train_pred)

    run_count += 1

print(f"Train accuracy on {folds}-fold cross-validation: {train_total / float(run_count)}")
print(f"Test accuracy on {folds}-fold cross-validation: {test_total / float(run_count)}")

pol_output["test"] = [(test_total / run_count)]
pol_output["train"] = [(train_total / run_count)]
pol_output = pd.DataFrame(pol_output)

print("Average confusion matrix:")
pol_matrices = np.mean(np.array(pol_matrices), axis=0)
print(pol_matrices)

pol_fpr = np.mean(np.array(pol_fpr), axis=0)
pol_tpr = np.mean(np.array(pol_tpr), axis=0)
pol_thresh = np.mean(np.array(pol_thresh), axis=0)

pol_auc = pol_auc / float(run_count)




pol_output

0
tf_idf training
tf_idf testing
ensemble fitting
1
tf_idf training
tf_idf testing
ensemble fitting
2
tf_idf training
tf_idf testing
ensemble fitting
3
tf_idf training
tf_idf testing
ensemble fitting
4
tf_idf training
tf_idf testing
ensemble fitting
Train accuracy on 5-fold cross-validation: 0.8892
Test accuracy on 5-fold cross-validation: 0.6732000000000001
Average confusion matrix:
[[335.4 156.4]
 [170.4 337.8]]


,train,test
0,0.8892,0.6732


In [7]:
'''
    TF-IDF
'''
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import KFold


# average accuracies
tf_output = {"train" : None, "test" : None}
tf_matrices = []

train_total = 0
test_total = 0
run_count = 0

tf_fpr = []
tf_tpr = []
tf_thresh = []

tf_auc = 0

#lbgfs
x = features
y = classes
folds = 5
kf = KFold(n_splits=folds)

for i, (train_index, test_index) in enumerate(kf.split(x)):
    print(i)
    scaler = StandardScaler()
    print("tf_idf training")
    tf_idf_training, names, vectorizer = get_tf_idf_vector(filtered["corpus"].iloc[train_index])
    x_train = pd.concat((x.iloc[train_index].reset_index(drop=True), tf_idf_training), axis=1).drop(features, axis=1)
    
    x_train = scaler.fit_transform(x_train)

    print("tf_idf testing")
    x_test = pd.DataFrame(vectorizer.transform(filtered["corpus"].iloc[test_index]).toarray(), columns=[f"__word{i}" for i in range(len(names))])
    x_test = pd.concat((x.iloc[test_index].reset_index(drop=True), x_test), axis=1).drop(features, axis=1)
    x_test = scaler.transform(x_test)
    
   
    
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]
    
    tf_ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(kernel='rbf', probability=True, random_state=42))
    ], voting='soft')
    print("ensemble fitting")
    tf_ensemble.fit(x_train, y_train)
    y_pred = tf_ensemble.predict(x_test)
    fpr, tpr, thresh = metrics.roc_curve(y_test, y_pred, pos_label=1)
    tf_fpr.append(fpr)
    tf_tpr.append(tpr)
    tf_thresh.append(thresh)
    tf_auc += metrics.auc(fpr, tpr)


    tf_matrices.append(confusion_matrix(y_test, y_pred))
    test_total += accuracy_score(y_test, y_pred)

    y_train_pred = tf_ensemble.predict(x_train)
    train_total += accuracy_score(y_train, y_train_pred)

    run_count += 1

print(f"Train accuracy on {folds}-fold cross-validation: {train_total / float(run_count)}")
print(f"Test accuracy on {folds}-fold cross-validation: {test_total / float(run_count)}")

tf_output["test"] = [(test_total / run_count)]
tf_output["train"] = [(train_total / run_count)]
tf_output = pd.DataFrame(tf_output)

print("Average confusion matrix:")
tf_matrices = np.mean(np.array(tf_matrices), axis=0)
print(tf_matrices)

tf_fpr = np.mean(np.array(tf_fpr), axis=0)
tf_tpr = np.mean(np.array(tf_tpr), axis=0)
tf_thresh = np.mean(np.array(tf_thresh), axis=0)

tf_auc = tf_auc / float(run_count)



tf_output

0
tf_idf training
tf_idf testing
ensemble fitting
1
tf_idf training
tf_idf testing
ensemble fitting
2
tf_idf training
tf_idf testing
ensemble fitting
3
tf_idf training
tf_idf testing
ensemble fitting
4
tf_idf training
tf_idf testing
ensemble fitting
Train accuracy on 5-fold cross-validation: 0.99535
Test accuracy on 5-fold cross-validation: 0.9301999999999999
Average confusion matrix:
[[453.6  38.2]
 [ 31.6 476.6]]


,train,test
0,0.99535,0.9302


In [8]:
'''
    combine accuracies into one table
'''

final_results = pd.concat((pol_output, tf_output, comb_output), axis=0)
final_results.index = ["Polarity", "Tf-Idf", "Polarity + Tf-Idf"]
final_results

,train,test
Polarity,0.88920,0.6732
Tf-Idf,0.99535,0.9302
Polarity + Tf-Idf,0.99605,0.9272


In [9]:
'''
    Finally, save accuracy metrics to the spreadsheet
'''




book = load_workbook(EXCEL_FILE)
writer = pd.ExcelWriter(EXCEL_FILE, engine="openpyxl")
writer.book = book
final_results.to_excel(writer, sheet_name=f"accuracies")
book.save(filename=EXCEL_FILE)
book.close()



C:\Users\desmo\AppData\Local\Temp\ipykernel_15680\3534906820.py:10: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book


In [10]:
import openpyxl.drawing
from sklearn import metrics
import matplotlib.pyplot as plt
import io


plt.figure()
lw = 2
plt.plot(pol_fpr, pol_tpr,
 lw=lw, label='Polarity: (%0.2f)' % pol_auc)
plt.plot(tf_fpr, tf_tpr,
 lw=lw, label='Tf-Idf: (%0.2f)' % tf_auc)
plt.plot(comb_fpr, comb_tpr,
 lw=lw, label='Polarity + Tf-Idf: (%0.2f)' % comb_auc)
plt.plot([0, 1], [0, 1], color='black', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic (ROC)')
plt.legend(loc="lower right")
#plt.show()


# save figure as PNG
png = io.BytesIO()
plt.savefig(png, format="png")


# write PNG to excel file
book = load_workbook(EXCEL_FILE)
ws = book.active

img = openpyxl.drawing.image.Image(png)
img.anchor = "A1"
ws.add_image(img)
book.save(filename=EXCEL_FILE)
plt.close()
book.close()

